## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-15-22-47-00 +0000,wsj,The WSJ Dollar Index Falls 0.10% to 96.02,https://www.wsj.com/finance/currencies/asian-c...
1,2025-12-15-22-46-49 +0000,wapo,Brown University shooting live updates: Police...,https://www.washingtonpost.com/nation/2025/12/...
2,2025-12-15-22-46-00 +0000,wsj,Opinion | Why Trump Is Targeting Proxy Advisers,https://www.wsj.com/opinion/why-trump-is-targe...
3,2025-12-15-22-45-44 +0000,nyt,Harvard’s President Will Stay in the Role Inde...,https://www.nytimes.com/2025/12/15/us/harvard-...
4,2025-12-15-22-45-11 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/12/15/us/bro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
104,bondi,34
116,reiner,34
105,beach,29
124,rob,29
6,shooting,28


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
64,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...,157
160,2025-12-15-17-48-41 +0000,nypost,Bondi Beach father-son terror suspects travele...,https://nypost.com/2025/12/15/world-news/bondi...,156
143,2025-12-15-18-30-23 +0000,nypost,Officer learned he was going to be a dad days ...,https://nypost.com/2025/12/15/world-news/shot-...,152
114,2025-12-15-19-46-04 +0000,startribune,Rob Reiner's son Nick arrested after the direc...,https://www.startribune.com/family-member-ques...,137
43,2025-12-15-21-45-29 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,133


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
64,157,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...
160,156,2025-12-15-17-48-41 +0000,nypost,Bondi Beach father-son terror suspects travele...,https://nypost.com/2025/12/15/world-news/bondi...
1,96,2025-12-15-22-46-49 +0000,wapo,Brown University shooting live updates: Police...,https://www.washingtonpost.com/nation/2025/12/...
209,69,2025-12-15-15-27-09 +0000,nypost,FBI stops planned New Year’s Eve Los Angeles b...,https://nypost.com/2025/12/15/us-news/fbi-stop...
240,68,2025-12-15-12-02-50 +0000,bbc,Pro-democracy Hong Kong tycoon Jimmy Lai convi...,https://www.bbc.com/news/articles/cp844kjj37vo...
265,49,2025-12-15-10-30-00 +0000,wsj,"For months, a crucial question has been hangin...",https://www.wsj.com/economy/jobs/jobs-report-o...
87,47,2025-12-15-20-46-29 +0000,nypost,Singer Trey Songz tells cops he’s the victim a...,https://nypost.com/2025/12/15/us-news/singer-t...
46,40,2025-12-15-21-40-09 +0000,wapo,U.S. offers Ukraine security guarantee in bid ...,https://www.washingtonpost.com/politics/2025/1...
78,37,2025-12-15-20-55-31 +0000,nypost,Holocaust survivors and children among the 15 ...,https://nypost.com/world-news/bondi-beach-shoo...
297,35,2025-12-15-05-40-14 +0000,latimes,"Rob Reiner, 'When Harry Met Sally' director, '...",https://www.latimes.com/entertainment-arts/mov...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
